In [5]:
# from model import create_eva_vit_g
# model = create_eva_vit_g(precision="fp16").cuda()
# import torch
# with torch.cuda.amp.autocast():
#     inputs = torch.randn(5, 3, 224, 224).cuda()
#     result = model(inputs)
    



In [2]:
import json
from box import Box

with open('/home/huimingsun/Desktop/NGP/video_sgements/config/config.json', 'r') as f:
    config_dict = json.load(f)



config = Box(config_dict)

# Now you can access values in the config like this:
vit_model = config.model.vit_model
vit_model_path = config.model.vit_model_path
# ... and so on for the other values


In [17]:


import torch
from torch.cuda.amp import autocast as autocast
import torch.nn as nn
from eva_vit import Trans_Block
from recurrent_memory_transformer_pytorch import RecurrentMemoryTransformer

try:
    from .blip2 import Blip2Base, disabled_train
except:
    from blip2 import Blip2Base, disabled_train
from einops import rearrange

class VSeg(Blip2Base):
    """
    VideoChat model.
    """
    def __init__(self, config):
        super().__init__()


        self.low_resource = config.low_resource

        self.vit_precision = config.vit_precision
        print(f'Loading VIT. Use fp16: {config.vit_precision}')
        self.visual_encoder, self.ln_vision = self.init_vision_encoder(
            config.vit_model, config.img_size, config.drop_path_rate, 
            config.use_grad_checkpoint, config.vit_precision, config.vit_model_path,
            temporal_downsample=config.temporal_downsample,
            no_lmhra=config.no_lmhra, 
            double_lmhra=config.double_lmhra,
            lmhra_reduction=config.lmhra_reduction, 
            gmhra_layers=config.gmhra_layers, 
            gmhra_drop_path_rate=config.gmhra_drop_path_rate,
            gmhra_dropout=config.gmhra_dropout, 
        )
        if config.freeze_vit:
            print("freeze vision encoder")
            if not config.freeze_mhra:
                open_list = []
                for name, param in self.visual_encoder.named_parameters():
                    if 'mhra' not in name:
                        param.requires_grad = False
                    else:
                        open_list.append(name)

            else:
                for name, param in self.visual_encoder.named_parameters():
                    param.requires_grad = False
                self.visual_encoder = self.visual_encoder.eval()
                self.visual_encoder.train = disabled_train
                for name, param in self.ln_vision.named_parameters():
                    param.requires_grad = False
                self.ln_vision = self.ln_vision.eval()
                self.ln_vision.train = disabled_train
                
        self.RMT = RecurrentMemoryTransformer(
            num_tokens = 512,               # number of tokens
            num_memory_tokens = 128,          # number of memory tokens, this will determine the bottleneck for information being passed to the future
            dim = 512,                        # model dimensions
            depth = 6,                        # transformer depth
            causal = True,                    # autoregressive or not
            dim_head = 64,                    # dimension per head
            heads = 8,                        # heads
            seq_len = 512*10,                   # sequence length of a segment
            use_flash_attn = True             # whether to use flash attention
        )

        self.blocks = nn.ModuleList([
                Trans_Block(dim = 512, num_heads = config.embedding_size//88, mlp_ratio= 4.3637)
                    for i in range(10)])
        self.norm = nn.LayerNorm(config.embedding_size)
        self.fc_norm = nn.LayerNorm(config.embedding_size)
        self.fn1 =nn.Linear(1408,512)  #feature number from 1408 to 512
        self.fn2 = nn.Linear(1286,512) #patch number from 1286 to 512
        
        self.fn3 = nn.Linear(512*10,512)
        self.memory = None
        self.head = nn.Linear(512, config.frames)
        self.score_head = nn.Linear(config.embedding_size, 1)
        self.max_txt_len = config.max_txt_len
        self.cache_data = []

    def vit_to_cpu(self):
        self.ln_vision.to("cpu")
        self.ln_vision.float()
        self.visual_encoder.to("cpu")
        self.visual_encoder.float()

    def forward_all_features(self, interval_1,interval_2):

        with self.maybe_autocast():
            T = interval_1.shape[1]
            # use_image = True if T == 1 else False
            interval_1 = interval_1.permute(0, 2, 1, 3, 4) # [B,T,C,H,W] -> [B,C,T,H,W]
            interval_2 = interval_2.permute(0, 2, 1, 3, 4) # [B,T,C,H,W] -> [B,C,T,H,W]
            interval1_embeds = self.ln_vision(self.visual_encoder(interval_1))
            interval2_embeds = self.ln_vision(self.visual_encoder(interval_2))
        x = torch.concat((interval1_embeds, interval2_embeds), dim=1)
        return x


    def forward_feature(self, interval_1):

        with self.maybe_autocast( dtype=torch.float16):
            T = interval_1.shape[1]
            # use_image = True if T == 1 else False
            interval_1 = interval_1.permute(0, 2, 1, 3, 4) # [B,T,C,H,W] -> [B,C,T,H,W]
            interval1_embeds = self.ln_vision(self.visual_encoder(interval_1))
        return interval1_embeds

            
    def NoRMT_forward(self, interval_1,interval_2, y = None):
        
        x = self.forward_features(interval_1,interval_2)
        print(x.shape)
        x = self.norm(x)
        
        for block in self.blocks:
            x = block(x)
        print(x.shape)
        x = x[:, 0]
        x = self.head(x)
        if y!= None:
            loss_fn = nn.BCEWithLogitsLoss()
            loss = loss_fn(x, y)
            return x, loss
        else:
            return x     
        
        
    def RMT_forward(self, x):

        #[1, 512, 512]
        print('RMT inputL ', x.shape)
        RMT_output, self.memory, _ = self.RMT(x,self.memory)        # (1, 1024, 20000), (1, 128, 512), None
        return  RMT_output

    def clear_cache(self):
        self.cache_data = []
        self.memory = None
    
    def forward(self, interval_1, y = None):
        
        vit_feature = self.forward_feature(interval_1)
        vit_feature = self.norm(vit_feature)

        vit_feature = self.fn1(vit_feature)
        vit_feature = rearrange(vit_feature, 'b c l -> b l c') # (patch number, feature vector) -> ( feature vector, patch number)
        vit_feature = self.fn2(vit_feature)                    # decrease the patch number from 1286 to 512
        vit_feature = rearrange(vit_feature, 'b l c -> b c l' ) # ( feature vector, patch number) -> (patch number, feature vector) 
          


        if len(self.cache_data) == 0:
            self.cache_data.append(vit_feature)
            Prev_feature = torch.zeros_like(vit_feature)
        else:   
            Prev_feature = self.cache_data.pop()
            self.cache_data.append(vit_feature)

        RMT_feature = self.RMT_forward(vit_feature)

        x = torch.cat((RMT_feature, Prev_feature, vit_feature), dim = 1)
        # print('combined_feature: ', x.shape)
        for block in self.blocks:
            x = block(x)
        # print(x.shape)
        x = x[:, 0]
        x = self.head(x)
        if y!= None:
            loss_fn = nn.BCEWithLogitsLoss()
            loss = loss_fn(x, y)
            return x, loss
        else:
            return x     

In [18]:
import json
from box import Box

with open('/mnt/drive1/hsun/videoSeg/config/config.json', 'r') as f:
    config_dict = json.load(f)

config = Box(config_dict)
model = VSeg(config.model).cuda()
video = torch.rand(1, 5, 3,  224, 224).cuda()
output = model(video)
output = model(video)

print(output.shape)

Loading VIT. Use fp16: fp32
Temporal downsample: False
freeze vision encoder


current vit feature after linear : torch.Size([1, 512, 512])
RMT inputL  torch.Size([1, 512, 512])
RMT output:  torch.Size([1, 512, 512]) Prev_feature:  torch.Size([1, 512, 512]) vit_feature:  torch.Size([1, 512, 512]) Memory shape:  torch.Size([1, 128, 512])
combined_feature:  torch.Size([1, 1536, 512])
torch.Size([1, 1536, 512])


torch.Size([1, 8])

In [10]:
import torch

model = 

In [11]:
x = torch.randint(0, 256, (1, 512*10)).cuda()
print(x.shape)
# x = torch.randint(0, 256, ( 197, 768))
# print(x.shape)
logits1, mem, _ = model(x)        # (1, 1024, 20000), (1, 128, 512), None
print(logits1.shape, mem.shape)
logits1, mem, _ = model(x, mem)  # (1, 1024, 20000), (1, 128, 512), None
logits1, mem, _ = model(x, mem)  # (1, 1024, 20000), (1, 128, 512), None
print(logits1.shape, mem.shape)

# and so on ...


torch.Size([1, 5120])
torch.Size([1, 5120, 512]) torch.Size([1, 128, 512])
torch.Size([1, 5120, 512]) torch.Size([1, 128, 512])


In [8]:
import torch
from recurrent_memory_transformer_pytorch import RecurrentMemoryTransformer

model = RecurrentMemoryTransformer(
    num_tokens = 512,               # number of tokens
    num_memory_tokens = 128,          # number of memory tokens, this will determine the bottleneck for information being passed to the future
    dim = 512,                        # model dimensions
    depth = 6,                        # transformer depth
    causal = True,                    # autoregressive or not
    dim_head = 64,                    # dimension per head
    heads = 8,                        # heads
    seq_len = 512,                   # sequence length of a segment
    use_flash_attn = True             # whether to use flash attention
).cuda()

x = torch.randn((1, 512,512)).cuda()
# x  = torch.randint(0, 256, (1, 1024)).cuda()

# print(x.shape,x1.shape, x.dtype, x1.dtype)

# and so on ...


In [9]:
logits1, mem1, _ = model(x)        # (1, 1024, 20000), (1, 128, 512), None
logits2, mem2, _ = model(x, mem1)  # (1, 1024, 20000), (1, 128, 512), None
logits3, mem3, _ = model(x, mem2)  # (1, 1024, 20000), (1, 128, 512), None


In [10]:
logits3.shape

torch.Size([1, 512, 512])

In [12]:
mem2.shape

torch.Size([1, 128, 512])